# Data Cleanup: US Census Population Data

In this notebook, we'll turn raw 2020 US Census population data, downloaded as a CSV file, into a clean, ready-to-use DataFrame containing total population for all census tracts in the US and Puerto Rico. We'll then save this to Parquet format to reduce both file size and load time for use in subsequent notebooks.

#### *Data sources*

A variety of different routes can be taken to reach the same US census data, the specific steps I followed are detailed below. To save time and space, I stored the raw population data compressed as a TGZ file.

* 2020 Census: Redistricting File (Public Law 94-171) 
   - downloaded from the ["Table" tab](https://data.census.gov/cedsci/table?q=United%20States) on *Census.gov*
  - filters selected: Years: 2020 > Geography: Tract: All Census Tracts within United States > Topics: Populations and People: Populations and People
  - results list selection: "Decennial Census, P1 | RACE, 2020: DEC Redistricting Data (PL 94-171)" 

#### *Up close: data cleaning and mapping*

The notebooks below provide a detailed look at the other stages of cleaning and mapping the data for this project. The final steps of creating a SQL database of indoor farms in New York City and then mapping them will follow shortly.

* [Introduction: The State of Indoor Farming in the US](00_IndoorAgriculture_start.ipynb) 
<!-- * [Data Cleanup: US Census population data](01_DataCleanup_Population.ipynb)  -->
* [Data Cleanup: GEOID tables](02_DataCleanup_GEOIDs.ipynb)
* [Data Cleanup: Geographic data](03_DataCleanup_GIS.ipynb)
* [Mapping NY State: Population Density](04_Mapping_Population.ipynb)
* SQL Database Creation: Indoor Farms - *coming soon...*
* Mapping NY State: Indoor Farms - *coming soon...*

#### *Some helper code for managing file paths*
We'll use the code below throughout this project to make it easier to refer to the folders where our various data files are stored.

In [1]:
# the code in this cell uses the `os` and `pathlib` modules to make it easier to refer to 
# the various folders where data files for this project are stored
# everything builds off base_dir, so if we move our code later, we'll only need to change base_dir

# os module provides a variety of frequently used file system functions, including path.join
# pathlib module makes it easier to manipulate folder and file paths with Python
import os, pathlib

# base_dir - the immediate parent folder of this notebook
# our data folders are found in here
base_dir = pathlib.Path(os.getcwd()).parent

# data_archive - compressed files are stored here
# these will be preserved in git
data_archive_dir = os.path.join(base_dir, "data_archive")

# cleaned data files are stored here
clean_data_dir = os.path.join(data_archive_dir, "clean")

# data_dir - large/numerous files will go here
# these will not be preserved in git!
# only files that can be recreated are stored here (e.g. downloaded or unpacked from 
# data_archive, or generated from a DataFrame)
data_dir = os.path.join(base_dir, "data")

# shapes_dir - folders containing shapefiles go here
shapes_dir = os.path.join(data_dir,"shapes")

# json_dir - GeoJSON files we generate and want to save for re-use go here
json_dir = os.path.join(data_dir,"geojson")

# store here any special utility files we might want to reuse
util_dir = os.path.join(data_dir,"util")

## What are we looking at here?

Let's load our population data as a pandas DataFrame. Since our data is stored as a TGZ file, we'll use the `extract_from_tgz` function below to open it. If we were working with a regular non-compressed CSV file, we'd skip this step and simply open the file as a pandas DataFrame using `pd.read_csv`.

Using `.shape`, we can tell before even setting eyes on it, that our DataFrame is enormous - 73 columns and 85,000+ rows! Handily, the [Tallies list on Census.gov](https://www.census.gov/geographies/reference-files/time-series/geo/tallies.html#tract_bg_block) tells us the total number of ensus tracts for the US and Puerto Rico is 85,395. The count provided by `.shape` indicates one additional row, 85,396 total, but we'll be seeing shortly why that's ok.

In [2]:
# population data has been stored compressed as TGZ files to save time and space
import tarfile
import pandas as pd

# these options determine how much data is displayed in the notebook
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 32mb+ of census data saved in a 4.7mb archive
file_census_data_tgz = os.path.join(data_archive_dir, 'census_data_2022_03_01.tgz')

# this is the population data file we'll extract from the above tgz file
file_census_data_csv = 'DECENNIALPL2020.P1_data_with_overlays_2021-12-02T121459.csv'

# this function creates a DataFrame from our tgz archive file
def extract_from_tgz(filename):
    with tarfile.open(filename) as tf:
        for file in tf.getmembers():
            if file.name == file_census_data_csv:
                data = tf.extractfile(file)
                return pd.read_csv(data, low_memory=False, usecols=[0, 1, 2])

# now call the function to extract our tgz file and load it as a pandas DataFrame
df_census_pop = extract_from_tgz(file_census_data_tgz)

df_census_pop.shape

(85396, 3)

We can get a pretty good sense of the actual data by using `.head()` and `.tail()`to view just the first and last few rows.

In [3]:
df_census_pop.head(3)

,GEO_ID,NAME,P1_001N
0,id,Geographic Area Name,!!Total:
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055


In [4]:
df_census_pop.tail(3)

,GEO_ID,NAME,P1_001N
85393,1400000US72153750503,"Census Tract 7505.03, Yauco Municipio, Puerto Rico",2155
85394,1400000US72153750601,"Census Tract 7506.01, Yauco Municipio, Puerto Rico",4368
85395,1400000US72153750602,"Census Tract 7506.02, Yauco Municipio, Puerto Rico",2587


## Making things a little nicer

We can see that our columns have been named using codes that aren't terribly meaningful to us. The first table row, however, contains descriptions of the data that are a bit more helpful. Let's go ahead and drop the column headers and replace them with descriptions in the first data row. (This now make our total row count 85,395, which, as mentioned above, matched the total count of census tracts provided on the [Tallies list on Census.gov](https://www.census.gov/geographies/reference-files/time-series/geo/tallies.html#tract_bg_block).

In [5]:
df_census_pop = df_census_pop.drop(0)

We also want to specify that our first column contains the full 20-character GEOIDs used in CSV files downloaded from *data.census.gov*, rather than the shorter form 11-digit GEOIDs used in the Tiger/Line shapefiles we'll be looking at in the next notebook. A detailed explanation of [how GEOIDs work](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html), can be found in this [helpful article](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html) on *census.gov*.

In [6]:
df_census_pop.columns = ["GEOID Census Tract Full", "Census Tract Name", "Population"]
df_census_pop.head(3)

,GEOID Census Tract Full,Census Tract Name,Population
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216


## Rooting out missing values

Now that our columns names are all set, we can begin addressing some common data issues that might create problems later, such as missing/null values. Let's use`.info()` to check for these now.

In [7]:
df_census_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85395 entries, 1 to 85395
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   GEOID Census Tract Full  85395 non-null  object
 1   Census Tract Name        85395 non-null  object
 2   Population               85395 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB


According to `.info()`, our columns are free of null values. We *really* want to make sure this is the case, so that we can avoid the complications null once can create once we start working with our data. Let's probe a bit more, using `.isna()` to return a boolean indicating whether the observation in each column is missing, `True`, or not, `False`. 

In [8]:
df_census_pop.isna()

,GEOID Census Tract Full,Census Tract Name,Population
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
...,...,...,...
85391,False,False,False
85392,False,False,False
85393,False,False,False
85394,False,False,False


So far, so good. All rows are returning `False`, indicating no missing values. Let's use `.sum()` to get a full count of missing values, just in case some are lurking deeper in the data, out of sight in our current display.

In [9]:
df_census_pop.isna().sum()

GEOID Census Tract Full    0
Census Tract Name          0
Population                 0
dtype: int64

Good - still seeing 0 missing values. As a final check, lets tell pandas to actually *show us* any rows with missing values. 

In [10]:
df_census_pop[df_census_pop.isna().any(axis=1)]

,GEOID Census Tract Full,Census Tract Name,Population


Again, nothing! So, it looks like we are in the clear - no missing values to address. On to the next step. 

## Do our datatypes make sense?

We can call `.dtypes` on our DataFrame to check the datatypes for each column.

In [11]:
df_census_pop.dtypes

GEOID Census Tract Full    object
Census Tract Name          object
Population                 object
dtype: object

Hmm...something isn't quite right here. Based on our initial visual inspection we know that "Population" is an integer value, but `dtypes` is telling us it's data type is "object". Let's fix this by setting the datatype to, `int`.

We know that both "GEOID Census Tract Full" and "Census Tract Name" are strings. Pandas uses the data type `object` to store string values, so this means the data types for these columns are beings correctly reflected when we run `dtypes`.

In [12]:
df_census_pop['Population'] = df_census_pop['Population'].astype(int)
df_census_pop.dtypes

GEOID Census Tract Full    object
Census Tract Name          object
Population                  int64
dtype: object

In preparation for future steps in our data journey, we'll add a new column for "GEOID Census Tract" to our *df_census_pop* DataFrame. We'll be using this later to match our population data with our GIS data. We know that the last 11 digits of "GEOID Census Tract Full" represent the census tract level, so we can use a lambda to grab these digits and place them in our new column and set its data type to `int`.

In [13]:
df_census_pop['GEOID Census Tract'] = df_census_pop['GEOID Census Tract Full'].apply(lambda x: str(x)[-11:]).astype(int)

## A nice, tidy DataFrame!

There we have it - a clean, tidy, ready-to-use DataFrame of population data for every census tract in the US and Puerto Rico. Before we move on, we'll save it to our *Clean Data* directory, so it will be readily available when we need it.

In [14]:
df_census_pop.head(3)

,GEOID Census Tract Full,Census Tract Name,Population,GEOID Census Tract
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775,1001020100
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055,1001020200
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216,1001020300


In [15]:
# clean_census_pop_file = os.path.join(clean_data_dir,'census_pop.csv')
# df_census_pop.to_csv(clean_census_pop_file)

In [18]:
# let's save as arrow/parquet and see if it's a smaller file with dtypes preserved
import pyarrow as pa
import pyarrow.parquet as pq

census_table = pa.Table.from_pandas(df_census_pop, preserve_index=False)

clean_census_pop_file = os.path.join(clean_data_dir,'census_pop.parquet')
pq.write_table(census_table, clean_census_pop_file, compression='BROTLI')
census_table.schema

GEOID Census Tract Full: string
Census Tract Name: string
Population: int64
GEOID Census Tract: int64
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 604

In [17]:
# # this is a test to see if df loads ok from parquet file
# # it does!
# import dask.dataframe as dd
# df_census_test = dd.read_parquet(clean_census_pop_file)
# df_census_test.head()